In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import calendar
from pathos.multiprocessing import ProcessingPool as Pool
import openpyxl
#from multiprocessing import Pool
from time import sleep

start_time = time.time()
cal = calendar.Calendar()
result_list=[]
new_dict=[]

query = "여성"

def scrape(url):
    import requests
    from bs4 import BeautifulSoup
    try:
        res = requests.get(url, headers={'User-Agent':'Mozilla/5.0'}, allow_redirects=False)
        return res.text
    except:
        return "no text"

if __name__ == '__main__':
    
    for month in range(1, 13):
        monthdays = [d for d in cal.itermonthdays(2018, month) if d != 0]
        for day in monthdays:
            s_date = "2018." + str(month).zfill(2) + "." + str(day).zfill(2)
            e_date = s_date 
            s_from = s_date.replace(".","")
            e_to = s_from
            print(s_date)

            xlsx_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#1_2018_여성', '{}.xlsx'.format("#1_"+s_from+"_여성"))
            wb = openpyxl.Workbook()
            sheet = wb.active
            sheet.append(["title", "source", "date", "html"])
            wb.save(xlsx_name)
            
            book = openpyxl.load_workbook(xlsx_name)
            writer = pd.ExcelWriter(xlsx_name, engine='openpyxl')
            writer.book = book
            writer.sheets = {ws.title: ws for ws in book.worksheets}            

            for n in range(1, 4001, 10):
                url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=1&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(n)
                raw = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
                html = BeautifulSoup(raw.text, "html.parser")
                articles = html.select("ul.type01 > li")

                if not articles:
                    break
                
                aid = list()
                title = list()
                urls = list()
                source = list()
                date = list()
                records = list()
                html = list()
                
                for ar in articles:
                    
                    aid.append(ar.select_one("a").get('asplugin-id'))
                    title.append(ar.select_one("a._sp_each_title").text)
                    urls.append(ar.select_one("a").get('href'))
                    source.append(ar.select_one("span._sp_each_source").text)
                    dt = str(ar.select_one("span.bar").next_sibling)
                    if dt:
                        date.append(dt)
                    else:
                        date.append(s_from + ".")
                
                with Pool(10) as p:
                    records = p.map(scrape, urls)
                
                for re in records:
                    html.append(BeautifulSoup(re, "lxml"))
                
                print(title)
                d = pd.DataFrame({"aid": aid, "title": title, "source": source, 
                                  "date": date, "html": html, "url": urls}) 
                sleep(0.001)
                
                for sheetname in writer.sheets:
                    try:
                        d.to_excel(writer,sheet_name=sheetname, startrow=writer.sheets[sheetname].max_row, index = False,header= False)
                    except:
                        print("error")
            writer.save()
                
print("--- %s seconds ---" % (time.time() - start_time))

2018.01.01
['독일서 새해맞이 폭죽놀이 중 2명 사망', '신진 과학자 지원 대폭 늘린다…과기정통부 2018 연구개발 사업 종합시행계획...', '담뱃불 부주의 3자녀 숨지게 한 22세 엄마 구속영장', '독일서 새해맞이 폭죽놀이 중 2명 사망', '[데스크 칼럼]‘58년 개띠’의 2018년 새해', '산청군, 작년 20차례 우수 지자체 선정', '[신년사] 허태정 대전 유성구청장 "미래형 명품 자족도시 실현"', "창녕군, 농촌 여성 생활기술교육 '인기'", '1년간 고성 군민 인명ㆍ재산 보호 보람 느껴', '일본군 위안부 재협상 당사자 의견 반영을']
['[강상삼 소장의 카사노바에게 배우는 퍼펙트 세일즈<68>]시장은 찾지 않고 만...', '창녕군, 내년부터 공중화장실 휴지통 없앤다', '"새해부터 예쁨"···표예진, 러블리 잔뜩 묻은 미모', '[기자의시각] 함께 다사다난 합시다', '여성농업인 거제시연합회 행복천사 66호 지정', '[신년기획] 경남도지사 누가 뛰나', '김해시장 신년사', "[우리모두뉴스] 바람만 불어도 아픈 '통풍'을 아세요?", "[우리모두뉴스] 여성가족부는 '인터넷 속에서도 남자와 여자가 똑같이 대접받...", '"대통령 문재인입니다"…새해 \'깜짝 전화\'']
['제천 참사 유족 "건물 내부에 있던 희생자와 1시간 2분 15초 통화"', '치과 치료 불만…5일간 43차례 전화 70대 여성 벌금형', '[지방선거 여론조사] 민주당 부산시장 후보군 선호도', '[지방선거 여론조사] 한국당 부산시장 후보군 선호도', '[지방선거 여론조사] 민주당 경남지사 후보군 선호도', '[지방선거 여론조사] 한국당 경남지사 후보군 선호도', '[1분뉴스] 양산 추돌사고 뺑소니범, 16시간 만에 자수', '낸시랭 "서로 사랑하게 해주세요"..왕진진과 밝게 인사', "[지방선거 여론조사] '현역 프리미엄' 권영진 시장 한국당 내 선두", '[지방선거 여론조사]민주당 28.1% > 한국당 26.0%…대구서 정당지지도 앞서']
["[지방선거 

['[기고] 주민의 시각에 보여지는 치안행정', '[신년사] 이춘희 세종시장', '문재인 대통령, 새해 첫날 해돋이 산행..."국민 삶 나아지길"', '국민소득 3만불 시대 개막… 분배로 삶의 질 높인다', '[신년사] 황선봉 예산군수', '홍준표·안철수·유승민, 靑 신년인사회에 불참 결정', '[기초단체장 누가 뛰나-부산] 16개 구·군 8 대 1경쟁률… 남구청장 14명 도전...', '[fn이사람] 정인숙 한우리 독서지도사 "경단녀에서 독서지도사로 새 삶"', '쌀로 만들어 식감 높인 메뉴부터 칼로리와 영양을 고려한 건강한 쌀 디저트까...', '대구가톨릭대병원, 생체 간이식 수술 500례 달성']
['"반짝 뜨니 더 북적이네" 가구도 팝업스토어 열풍', '[아직 살만한 세상] 아무도 없는 ‘어두컴컴한 밤’에 도움의 손길 건넨 노숙...', '文 대통령, 이승엽 선수ㆍ포항 지진피해자ㆍ미혼모 등 13명과 ‘깜짝’ 신년 ...', "'제천 화재 참사' 수사 제천시·소방서 등 관공서로 확대", "문 대통령, 의인들과 신년맞이 '해돋이 산행'…시민과 '깜짝 통화'", "문 대통령, 의인들과 신년맞이 '해돋이 산행'…시민과 '깜짝 통화'", '검찰, 살인죄 구형 대폭 높인다…아동 납치살인 등 최대 사형', '[인물수첩] 이오숙 대구 북부소방서장', '[대구경북 인사·모임·화촉·부음] 1일', '대구의 애국 향한 위대한 정신… 무술년 전국 대표도시로 도약 이끈다']
['˝내가 지역 발전 이끌 적임자˝ 6·13 지선 포항시장 후보 `각축`… 이강덕 ...', '대구시, 택배기사 사칭 등 범죄 원천차단 원룸 밀집존`무인방범택배함`설치·...', '프로필-이오숙 대구북부소방서장', '경북도, 도청신도시 활성화 사업 `속도 낸다`', '대가대병원, 생체 간이식 수술 500례 달성', '대구국제공항 이전 핫이슈…보수텃밭 격변여부 주목', '황금개띠 기운 받아 안방·스크린 넘나들며 달린다', '지메르만·키신·페라이어… 클래식 마니아 ‘운명같은 해’', '[알림] 새 필진이 찾아

['문 대통령, 신년 맞아 국민과의 전화 통화', '화장실 휴지통 사라지고, 노후경유차 운행 제한된다', '문 대통령, 신년 맞아 국민과의 전화 통화', "'대통령 문재인 입니다' 국민과의 전화 통화", '흑백필름과 한지로 담아낸 여성의 속내', '[최형미의 다시 만난 세상] 여성주의는 새로운 사유방식이 필요하다', '[여성논단] 성평등 가치와 패러다임 안착되는 첫 해로', "222만 달러 두고 워렌 버핏 vs 헤지펀드 '세기의 대결'…버핏 압승", '문 대통령, 신년 맞아 국민과의 전화 통화', "'대통령 문재인 입니다' 국민과의 전화 통화"]
['[신년사] 전종한 천안시의장', '(경력채용과) 2017년도 5급 민경채 합격자 발표', '"새해 금연 결심 하셨나요?"', '진천 민주평통 여점숙 자문위원 민주평통의장(대통령) 표창 수상', '지하상가 매장의 비밀', "충주 자랑 향토가수 '김보겸 콘서트' 대성황", '충북도 여성정책관에 박현순 청주시청소년상담복지센터장 내정', '이시종 충북지사 신년사, 망원진세(望遠進世)의 한 해로 ... 충북미래비전 및...', '[여명] 바꿔야 할 미래 연대기', '살아나는 소비심리 꺼질라…백화점 화끈한 신년 할인전']
['전국 공중화장실 휴지통 사라진다…해외체류신고 인터넷으로 가능', "'지드래곤의 ♡' 이주연, 10년전 벌인 '90일녀' 퍼포먼스의 정체는?", '“겨울철, 다이어트하기에 가장 적절한 계절”', '“북한, 제재 후 실질적 변화… 실업률 상승ㆍ인플레이션 위험”', '[공감이슈] ‘문재인 정부 첫 특별사면의 대상과 내용은?’', '신세계 여주·파주·시흥·부산 프리미엄 아울렛, 1월7일까지 ‘새해 첫 세일...', '대전 동구 주택서 불...1명 사망', '대전 동구 주택서 불...1명 사망', '“한인사회 성장 원동력 되겠습니다”', '상주시 인사']
['정려원, 못다한 수상소감 “성범죄 피해자들 목소리 높일 수 있는 사회 되길”', '윤정수X양세형X한은정X써니 ‘현실남녀’, 오는 11일 첫 방 “남녀 고정관

['취객 하차요구 무시 후 목적지 주행한 택시…法 "감금 아냐"', '2018년 과기정통부 R&D사업…연구자중심·미래대비 ‘방점’', '장애인고용공단, 비정규직 173명 정규직 전환 완료', '과기정통부, 4.7조 규모 R&D 종합계획 확정', '공중화장실 휴지통 없앤다…새해 달라지는 제도는', '[6·13지선:횡성] 민주당·한국당 양당구도…한규호 출마 미지수', '한국공인노무사회 제17대 회장에 박영기씨 취임', '국가 R&D 4.6조원 투입..혁신성장동력 육성 목표', '2018년 행정제도 이렇게 달라집니다', '장애인고용공단, 비정규직 173명 정규직 전환']
['과기정통부 R&D 사업 "연구자 자율강화·혁신성장분야 지원"', '올해부터 민방위훈련 연4회 확대…공중화장실 휴지통도 사라져', '장애인고용공단, 비정규직 173명 1일자로 정규직 임용', '[6·13지선:원주]민주당vs한국당 양당구도…당내 경쟁 치열', '무술년 국가 R&D 투자 ‘미래·기초·신진·사회문제’ 방점', "[6·13지선:인천]'부채 프레임' 이번에도 먹힐까", '장애인고용공단, 비정규직 근로자 173명 정규직 전환', '전국 123개 자원봉사센터 민영화…부처별 프로그램 개발', "한지민·수지·설리…'82라인-94라인'의 뷰티 팁", '메가젠임플란트, 일자리창출 유공정부포상 국무총리표창 수상']
["류화영, 여성미+성숙된 매력으로 '시선 강탈'", '(재)전북여성교육문화센터 제6대 센터장 이윤애 전 전북해바라기센터 부소장', '[인도에서 온 런치박스]“모두가 행복한 2018년이 되길“···13억 인도의 새...', "문 대통령, 새해 첫날 의인 6명과 '해맞이 산행'", '광주·전남 장기미제 강력사건 아직 17건 남아…드들강 여고생 살인사건 해결...', '[신년사] 황숙주 순창군수', '[천지일보 신년여론조사] 국민 64.7% "적폐청산 수사 계속"', '서울소방공무원 역사상 최초 소방서장급4급 오른 이원주씨', '文대통령 무술년 첫 일정, 6명의 義人과 ‘해맞이 등산’', '"무

["[6·13지선:경남] 민주당 '탈환' vs 한국당 '수성'", '만평]영남권 첫 여성소방서장 탄생', '「2018년도 과학기술· ICT 분야 RD사업 종합시행계획」 확정', "[투데이 연예 톡톡] '2017 MBC가요대제전' 최정상 가수들 특급 무대 外", '文 대통령, 해맞이 산행으로 새해 시작', '러시아, 3월 대선·6월 월드컵 개최', '사고 잇따른 2017년…안전 의식 개혁해야', '[인터뷰①] 정려원 "`마녀의 법정` 성범죄 묵직한 주제에 시청률까지 따라 계...', '[사람들] "이름 때문에 완도 자원…정년까지 지킬 것" 김완도 순경', '[신년사] 유정복 인천시장 “재정건전화 성과 시민 품으로”']
['이주의 쇼핑정보(2018년 1월 1일 ~ 1월 7일)', '‘M&A 시장 활성화’‧‘대기업-벤처 오픈 플랫폼’ 등 벤처업계 꿈의 과제 시...', '탈모샴푸 전문 브랜드 노타모5.5...신년맞이 추가 증정 등 풍성한 혜택 누린다', '신생아 태운 사설 구급차, 음주운전 차량과 충돌…4명 부상', '[비바100] 2018드라마 전망, #물량공세 #톱스타 경쟁 #스타작가 각축', '사회교육중앙회, 무료지원이벤트로 사회복지사자격증·학점은행제 수강희망자...', '"\'안나 카레니나\'는 모스크바적인 뮤지컬…광활하고 인상적"', "황금개의 해 'GOLDEN DOG'로 풀어본 식품업계 트렌드", '[주간한국][백운비 사주풀이] 결혼 앞두고 신체적 결함으로 고민', '[새해에는] 유근기 곡성군수 "농촌 활성화 최선"']
['[굿바이 빅뱅②]대마초 탑 언급부터 고무신 당부까지, 고척돔 울린 말말말', '[굿바이 빅뱅①]군대가면 끝? 13년차 빅뱅의 2막 기대되는 이유', "[카드뉴스] 2018 무술년 '황금 개띠' 정·관·재계 인사는 누구?", '[혁신성장, 창업의 길] 4대 벤처 협회장, 올바른 ‘벤처 생태계 구축’을 위해...', '[2018 유망주]무술년 예능, 조세호·김생민·박나래·장도연에 ‘올인’', "[스토리S] 2017 최다 클릭 '머슬퀸' B

...

["[창직이 미래다] '2017 창직어워드' 7개팀 영예", "[혁신으로 기업강국 이노베이션 Korea] 혁신으로 ‘기업강국'", '음주 택시가 차량 추돌 후 보행자 들이받아 1명 사망', '[새해에는] 김성 장흥군수 "대한민국 건강 1번지로"', '[S-girl] 레이싱 모델 윤희성, 22인치 허리-섹시 라인의 그녀', '[2018년 한국경제 대예측]<일자리>공공부문은 ‘개선’, 민간은 ‘글쎄’…호...', '고속도로 뺑소니, 새벽기도 일가족 참변 5명 사상', '부산 기장군 삼각산에 산불…음주운전으로 1명 숨져', '[새해에는] 강진원 강진군수 "농업소득 배가 원년 해로"', "[투데이 연예 톡톡] 전현무, 대상 이어 '예능인 브랜드 평판 1위'"]
['박영기 노무사, 한국공인노무사회 제17대 회장 취임', '류한우 군수, 체류형 문화관광 중심도시 도약 최선', '빚 갚으랴, 교육비 대랴 힘든 30ㆍ40대 …빈곤율 동반상승', '연예인 옷 24시간 대기…“하지만 알바만큼도 못받아요”', '[날씨와 생활 정보] 겨울철 운동 방법', '“여성 비만은 저소득층에, 남성은 고소득층에 다수”', '백화점업계, 새해 첫 영업일부터 ‘신년세일 경쟁’…롯데·현대·신세계·갤...', "살인죄 구형 높인다…아동 납치살해·강간 살해 최대 '사형' 구형", '부산 기장군 삼각산에 산불…음주운전으로 1명 숨져', "[어게인TV]'투깝스' 조정석, 진범 잡고 김선호 영혼 살려낼까"]
['文대통령, 오늘 각계 각층 인사 초청 신년 인사회', '성인병·국가암 이동검진 받으세요!', '新 年 辭', '횡단보도 건너던 보행자 덮쳐…1명 사망 外', "제약산업 스마트폰 '면역항암제', 어디까지 왔나", '"얼굴 하얘지는 약 주세요"…간염일반약 이상 품귀', 'AZ, ‘아타칸’ ‘아리미덱스’ 등 美 발매권 매각', '문 대통령, 각계 인사 청와대 초청해 신년 인사회', '[일자리가 희망이다]일자리 창출 해법은?…‘규제·노동개혁’ Vs ‘재벌개혁...', '알아두면 쓸데있는 ‘2018 황금 개

['익산여성회관, 강사 36명 공개모집', '휴지통 사라진 공중화장실, 적응 안된 시민들', '[리뷰]‘겟잇뷰티’ TOP3, allure 커버 촬영에 긴장감 상승', '공민지와 소향, 유럽의 중심에서 ‘복음의 얼굴’이 되다', "日정치권, 선거 후보자 '남녀 균등' 법안 재추진", '인하대병원 인천금연지원센터, 지역금연 민간보조사업 수행기관 선정', '도내 일부 건조주의보 크고 작은 화재 잇따라… 광명선 60대 여성 사망', '지하철역 화장실 등에서 104명 여성 신체 몰래 촬영한 30대, 집행유예', '‘한대앞 에이즈 감염 여성을 아시나요?’ 난데없는 괴담에 안산 일대 ‘공포...', '[인사] 인천서구']
['새해맞이 세일… ‘소비심리’ 불씨 지피기', '“불필요한 상고 줄인다”… 수원지검 형사상고심의위 출범', '市, 새해 ‘시정개혁’ 고삐… 종합감사 ‘적폐청산’ 정조준', '부천시여성단체협의회, 이웃돕기 성금 기탁', '구리 인구 20만명 돌파 도시 브랜드 가치 상승', '[인사] 안양시', '‘차량 테러 방지’ 방어벽·기둥 세우는 뉴욕', '“유명 그림책 전집 100권 중 99권…엄마를 앞치마 한 주부로만 그려”', "청주여성의전화 '성폭력 전문상담원' 교육 접수", '의학칼럼-방치하지 말고 완치하자, 요실금']
['분실된 대학원 입학시험 답지 가져간 60대 입건', "[TV온에어] '하하랜드' 떠돌이 견 바우, 긴 사투 끝 구조 성공", '[에너지경제신문 오프라인] 먼저 만나는 에너지경제신문 헤드라인 - 1월 4일', '유흥가에 문화예술거리 조성, 무슨 조화?', '미 중부 한파로 11명 사망…남동부로 확대', '<유통가 소식>', '"파리바게뜨, 제빵사 직접고용 안 하는 이유는…"', '충남 여성계 신년교례회', '베트남서 30대 한국인, 현지 여성 살해 혐의로 체포', '유족대책위 "골든타임 녹취분 미공개… 낱낱이 밝혀라"']
['울산YWCA, 다함께 시무식 가져', '그들은 다 어디로 갔나', '[이런생각]성평등, 인식의 전환이 필요하다', '

['[이형석의 창업의 비밀] 배낭여행과 창업 절차의 공통점(1)', '마틸다 효과', '개그맨 김형인 결혼 "9세 연하 일반인 여성과 3월 웨딩마치"', '니나', '루터를 통해 루터를 넘자', '충주 에네스티화장품, 임상실험 입증', '서초구, 엄마·아기 위한 특화 보건소 열어', '박근혜 정부 굴욕적 위안부 협상 원점서 다시 해야', '전문성·안목 갖춘 최고 필진들 한국불교 지성문화 이끈다', '‘의원 절반 여성’ 아이슬란드, 세계 첫 성평등 임금법']
['"기다리는 이들 있으니 멈출 수 없죠"', '[티브로드뉴스][중부]"일자리와 복지를 한 곳에서"..아산 고용복지+센터 개소', '블랙넛, 성희롱 가사 쏟아내다 결국 재판에…키디비 "물러서지 않을 것"', "'면역력↓' 공포의 통증이 엄습한다", '배우 신혜선, 과감한 어깨 노출...여성여성하다', '프랑스서 스마트폰 충전하면서 목욕하다 감전사', '신지애 선수 2년 연속 중앙자살예방센터에 기부', '‘동일임금 동일노동’, 아이슬란드서 법제화', '오스트리아에 \'반기문센터\' 개원…"청년·여성 권익신장"', '올해 TEC콘서트 첫 포문 … ‘4차 산업혁명과 여성리더’']
['그룹 바이브, 여성 관객 조롱 논란에 공식 사과', '2018년 농식품 분야 이렇게 달라진다.', '동양생명, (무)수호천사알뜰한통합GI보험(저해지환급형) 출시', '[유레카] 북간도에서 다시 만난 ‘동주’ / 이창곤', "전세계 독자 '페란테 열병' 빠트린 나폴리 4부작 완간", '노블스토리 안산점 정용순 원장 “올바른 임산부마사지와 산후관리가 손목통증...', '‘스노우맨’‘렛 미 인’에 이은 북유럽의 매혹적인 명품 스릴러 ‘오스’', "영동군, 가족친화인증 우수기관 '2년 더 유효'", '강진원 강진군수 "농업소득 배가 원년 해로"', '<레버넌트> 디카프리오 | 무성영화를 찍는 마음으로 ②']
['<레버넌트> 디카프리오 | 무성영화를 찍는 마음으로 ②', '[문화 生] 뮤지컬 \'카라마조프\' 이정수 "고전의 답, 현실에서 찾

["고혈압? '정맥 고혈압'도 주의 및 관리가 필요해", '경북도청 신도시 괄목 성장…상반기 인구 2만명 넘을듯', '[bnt화보] 이파니 “기획부터 대본까지 직접 참여한 웹툰 ‘조작’ 영화 계약...', '[bnt화보] 이파니 “노출 이미지? 어설픈 것보다 확실한 이미지 심는 게 낫다...', '[bnt화보] 이파니 “복코 없애려 코수술만 5번, 결국 의사도 포기해”', "소녀시대 윤아 , 'MBC 가요대제전' 때 입은 협찬의상 가격은?", '[경제단신] 기업은행, 무기계약직 3300명 정규직 전환', '잘못 배달 메뉴 환불거절에 흉기로 업주협박 40대 검거', '이파니 “시부모님과 관계? 아직 시간 필요한 단계”[포토엔]', "'엄지의 제왕' 고혈압, 발효쑥차의 효능 효과…규소청혈죽·규소소금 레시피"]
["평창 동계올림픽 성화, 5일 수원서 경기도 구간 봉송 '스타트'", '올림픽 방문객에 선물할 오륜복주머니', '[그러니까 낙태죄 폐지] ‘정상적’ 시민만 탄생시키려 한 국가, ‘낙태죄’ ...', '익산 여성들의 능력을 이끌 강사를 찾습니다！', '[bnt화보] 이파니 “실연당한 남편에 내가 먼저 대쉬, 어차피 만날 운명”', '[bnt화보] 이파니 “어렸을 때부터 만화가가 꿈, 웹툰작가 데뷔로 꿈 이뤄”', '[bnt화보] 이파니 “가슴 아픈 가족사, 언젠가 좋은 날 올 거라 믿어”', '속으로 끙끙, 치질 수술 피하려면…', '해바라기 로터리클럽, ‘지역 어르신 점심식사 대접’', '정보화 교육 신청하세요”']
['정보화 교육 신청하세요”', '출판사라면 누구나 탐내는 저자 1위', '탄수화물 다이어트…엄마가 아기 건강 망쳐', '성난 여드름, 여성의 아름다움에 한 몫하다', '약간의 과체중, 우울증 위험 낮고 오래 살아 (연구)', '카풀앱 드라이버 직접 해보니, 카카오택시 대안이 될 수 있을까', '[윤하정의 공연 세상] 2018년 뮤지컬, 연극 라인업', '채정호 “우리 삶의 달인은 나 자신”', '[아빠, 만화 안 그리고 뭐해?] 장애물 같은 

['[김맹녕의 골프 만평] 셀카는 콩글리시···셀피라고 말해요', '환경의 습격 ADHD, 배독숙면요법으로 근본치료 가능', '정현복 광양시장 "시민의 행복을 위해 혼심의 힘 다할 것"', '제천화재참사 현장 방문 유족들 "우리 말이 맞았다"', "광명시, 4차 산업혁명 시대 대비 전국 최초 '챗봇트레이너' 양성", '‘소년에서 숙녀로’ 어느 음악가의 30년 성전환 사진일기', "2018 아웃도어업계 트렌드 키워드는 'GOLDEN DOG'", '[곽도원 이사의 한의학이야기] 첩약 보험의 발의 그리고 한약 이야기', '낸시랭 부부, 경찰출동 소동 이유는?…“사실혼 여성 집 무단침입 추정”', '대전 기초의원 선거구 개편, 시민의 의견은?…5일 공청회 개최']
['임신에 어려움 겪는 부부들, 한방과 양방 융합 난임, 불임 치료가 도움 될 수...', '제주도, 2018년 도민사회교육 실시', "지적장애인 성폭행한 불법체류 태국인 '징역 5년'", '젊어지는 자궁경부암 환자...예방과 조기 발견이 최선', '기업 인사담당자 81% "채용시 성별 고려한다"', '6시간씩 3번 우려낸 사골국이 가장 맛있어', '지나친 음주와 기름진 음식, 통풍 발병률 높인다', '소음인 여성, 겨울철 수족냉증 주의해야…호르몬변화, 혈액순환 부족', "이혼전문변호사가 전하는 '이혼 소송에 대한 모든 것'", '[인터뷰] 이만종 한국테러학회 회장 “테러는 절망을 먹고 자란다”']
['원로·중견 ‘新作’ 러시… 문학계 더 뜨거워진다', '해냄여학생독학기숙학원 재수선행반 마감, 2월 재수생 대상 정규반 개강 예정', '모모랜드 "목표? 음원차트 100위 진입+해외 진출"', '[토픽] 버려진 ‘유기견’ 데려와 정성껏 돌봐준 가족에게 찾아온 행운', '기업 5곳 중 4곳 “채용할때 성별 고려한다”', "아이 키울 자격 없는 '불량 부모', 친권 박탈은 고작…", "[컬처톡] 비현실적이라 아름답고, 현실적이라 슬픈…연극 '블라인드'", '디스패치 "낸시랭 남편 전준주, 전자발찌 착용 맞다…동거녀

In [ ]:
 raw = requests.get("https://news.naver.com/main/list.nhn?mode=LPOD&mid=sec&oid=032&listType=title&date="+str(date)+"&page="+str(page),
                           headers={"User-Agent":"Mozilla/5.0"})

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import calendar
from pathos.multiprocessing import ProcessingPool as Pool
import openpyxl
#from multiprocessing import Pool
from time import sleep

start_time = time.time()

result_list=[]
new_dict=[]

query = "여성"

def scrape(url):
    import requests
    from bs4 import BeautifulSoup
    try:
        res = requests.get(url, headers={'User-Agent':'Mozilla/5.0'}, allow_redirects=False)
        return res.text
    except:
        return "no text"

if __name__ == '__main__':
    
    month = 1
    day = 1
    s_date = "2019." + str(month).zfill(2) + "." + str(day).zfill(2)
    e_date = s_date 
    s_from = s_date.replace(".","")
    e_to = s_from
    print(s_date)

    xlsx_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#1_2019_여성', '{}.xlsx'.format("#1_"+s_from+"_여성"))
    wb = openpyxl.Workbook()
    sheet = wb.active
    sheet.append(["title", "source", "date", "html"])
    wb.save(xlsx_name)

    book = openpyxl.load_workbook(xlsx_name)
    writer = pd.ExcelWriter(xlsx_name, engine='openpyxl')
    writer.book = book
    writer.sheets = {ws.title: ws for ws in book.worksheets}            

    for n in range(1, 4001, 10):
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=1&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(n)
        raw = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
        html = BeautifulSoup(raw.text, "html.parser")
        articles = html.select("ul.type01 > li")

        if not articles:
            break
            
        aid = list()
        title = list()
        urls = list()
        source = list()
        date = list()
        records = list()
        html = list()

        for ar in articles:

            aid.append(ar.select_one("a").get('asplugin-id'))
            title.append(ar.select_one("a._sp_each_title").text)
            urls.append(ar.select_one("a").get('href'))
            source.append(ar.select_one("span._sp_each_source").text)
            dt = str(ar.select_one("span.bar").next_sibling)
            if dt:
                date.append(dt)
            else:
                date.append(s_from + ".")

        with Pool(10) as p:
            records = p.map(scrape, urls)

        for re in records:
            html.append(BeautifulSoup(re, "lxml"))

        d = pd.DataFrame({"aid": aid, "title": title, "source": source, 
                          "date": date, "html": html, "url": urls}) 
        sleep(0.001)

        for sheetname in writer.sheets:
            try:
                d.to_excel(writer,sheet_name=sheetname, startrow=writer.sheets[sheetname].max_row, index = False,header= False)
            except:
                print("error")
    writer.save()

print("--- %s seconds ---" % (time.time() - start_time))

2019.01.01
--- 218.45918607711792 seconds ---
